# Train Tensorflow's Deep Multi-Layer Perceptron on MNIST dataset

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 28*28) / 255.0
X_test = X_test.reshape(-1, 28*28) / 255.0

In [3]:
from collections import namedtuple

NetworkTopology = namedtuple('NetworkTopology', 'inputs hiddens outputs'.split())

In [4]:
def he_initialization(n_inputs, n_outputs):
    stddev = 2/np.sqrt(n_inputs + n_outputs)
    return tf.initializers.truncated_normal(stddev = stddev, seed = 42)

In [5]:
def build_network(topology):
    X = tf.placeholder(tf.float32, shape=(None, topology.inputs), name='X')
    y = tf.placeholder(tf.int64, shape=(None), name='y')
    
    with tf.name_scope('dnn'):
        layer = X
        for idx, neurons in enumerate(topology.hiddens):
            init = he_initialization(int(layer.get_shape()[1]), neurons)
            layer = tf.layers.dense(layer, 
                                    neurons, 
                                    kernel_initializer = init,
                                    name='hidden'+str(idx), 
                                    activation=tf.nn.relu)
        logits = tf.layers.dense(layer, topology.outputs, name='outputs')
    
    with tf.name_scope('loss'):
        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
        loss = tf.reduce_mean(xentropy, name='loss')
    
    return X, y, logits, loss

In [6]:
%%time

from datetime import datetime
from sklearn.metrics import precision_score

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

tf.reset_default_graph()

topology = NetworkTopology(inputs = 28*28, hiddens = [256, 256, 128, 64], outputs = 10)

X, y, logits, loss = build_network(topology)

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
loss_summary = tf.summary.scalar('loss', loss)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 200
batch_size = 500

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(X_train.shape[0] // batch_size):
            start_index = iteration * batch_size
            end_index = start_index + batch_size
            X_batch = X_train[start_index:end_index]
            y_batch = y_train[start_index:end_index]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            if iteration % 10 == 0:
                summary_str = loss_summary.eval(feed_dict={X: X_test, y: y_test})
                step = epoch * (X_train.shape[0] // batch_size) + iteration
                file_writer.add_summary(summary_str, step)
        acc_train = accuracy.eval(feed_dict={X: X_train, y: y_train})
        acc_val = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Train accuracy:", acc_train, "Val accuracy:", acc_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

file_writer.close()

0 Train accuracy: 0.59678334 Val accuracy: 0.6113
1 Train accuracy: 0.7824 Val accuracy: 0.7949
2 Train accuracy: 0.84183335 Val accuracy: 0.8561
3 Train accuracy: 0.86883336 Val accuracy: 0.8773
4 Train accuracy: 0.8853667 Val accuracy: 0.891
5 Train accuracy: 0.89568335 Val accuracy: 0.9002
6 Train accuracy: 0.90285 Val accuracy: 0.9056
7 Train accuracy: 0.9087333 Val accuracy: 0.9097
8 Train accuracy: 0.9131167 Val accuracy: 0.9127
9 Train accuracy: 0.9170333 Val accuracy: 0.917
10 Train accuracy: 0.92065 Val accuracy: 0.9203
11 Train accuracy: 0.92321664 Val accuracy: 0.9228
12 Train accuracy: 0.92628336 Val accuracy: 0.9257
13 Train accuracy: 0.9284 Val accuracy: 0.9278
14 Train accuracy: 0.93078333 Val accuracy: 0.9303
15 Train accuracy: 0.9324333 Val accuracy: 0.9322
16 Train accuracy: 0.9343167 Val accuracy: 0.9336
17 Train accuracy: 0.9360667 Val accuracy: 0.9357
18 Train accuracy: 0.9378833 Val accuracy: 0.9369
19 Train accuracy: 0.93916667 Val accuracy: 0.938
20 Train accura

165 Train accuracy: 0.9932333 Val accuracy: 0.9734
166 Train accuracy: 0.9933 Val accuracy: 0.9735
167 Train accuracy: 0.9934 Val accuracy: 0.9735
168 Train accuracy: 0.9935167 Val accuracy: 0.9735
169 Train accuracy: 0.99365 Val accuracy: 0.9736
170 Train accuracy: 0.99373335 Val accuracy: 0.9737
171 Train accuracy: 0.9938167 Val accuracy: 0.9736
172 Train accuracy: 0.99395 Val accuracy: 0.9736
173 Train accuracy: 0.99408334 Val accuracy: 0.9737
174 Train accuracy: 0.99415 Val accuracy: 0.9737
175 Train accuracy: 0.9942333 Val accuracy: 0.9737
176 Train accuracy: 0.9943 Val accuracy: 0.9737
177 Train accuracy: 0.99448335 Val accuracy: 0.9737
178 Train accuracy: 0.99455 Val accuracy: 0.9738
179 Train accuracy: 0.9946333 Val accuracy: 0.9738
180 Train accuracy: 0.9947 Val accuracy: 0.9739
181 Train accuracy: 0.99471664 Val accuracy: 0.9739
182 Train accuracy: 0.99481666 Val accuracy: 0.9739
183 Train accuracy: 0.99486667 Val accuracy: 0.9742
184 Train accuracy: 0.99488336 Val accuracy: 